In [1]:
import os, glob, random, shutil, json

In [2]:
! ls /home/jovyan/nas-dataset/JUPR/MAM20044/Bad/*.png | wc -l
! ls /home/jovyan/nas-dataset/JUPR/MAM20044/Good/*.png | wc -l
! ls /home/jovyan/nas-dataset/JUPR/888/*.jpg | wc -l

7
17763
104


In [5]:
blackS = {
    "R3_MAM20044_1FDG71002398010E0AL00NU1_1603232235654.png",\
    "R3_MAM20044_1FDG71002398010E0AL00NUS_1603232235654.png",\
    "R3_MAM20044_1FDG71002398010E0AL00PLE_1603231217066.png",\
} # not really good (prevent good from referencing)

class Preprocess:
    def __init__(self, outputFolder, mode, imgPathL1, imgPathL2, activeJsonPath=None):
        """
        mode     imgPathL1                   imgPathL2 (always good)  
        + train: bad for validation (ood)  & good for train 
        + eval : c0  for unlabeled testing & c1   same as above
        activeJsonPath
        + 00: PD_B_GT_B, 
        + 01: PD_B_GT_G, add to trainset (FP) hard
        + 10: PD_G_GT_B, add to badList  (FN)
        + 11: PD_G_GT_G, add to trainset (TN) easy
        """
        global blackS
        if activeJsonPath:
            pathM  = json.load( open(activeJsonPath,"r") )
            blackS = blackS.union( { os.path.basename(path) for path in pathM[1][0] } ) # FN
            imgPathL2 += [ f"{os.path.dirname(imgPathL2[0])}/{os.path.basename(path)}" for path in pathM[0][1]+pathM[1][1] ] # FP, TN
        imgPathL2 = [ imgPath for imgPath in imgPathL2 if os.path.basename(imgPath) not in blackS ]
        for className,imgPathL in zip(['bad','good'] if mode=='train' else ['c0','c1'], [imgPathL1,imgPathL2]):
            folder = f"{outputFolder}/{mode}/{className}"
            os.makedirs(folder, exist_ok=True)
            for imgPath in imgPathL:
                shutil.copy(imgPath, folder)
            print( f"len(os.listdir({folder})) = {len(os.listdir(folder))}" )

In [6]:
bImgPathL = sorted(glob.glob("/home/jovyan/nas-dataset/JUPR/MAM20044/Bad/*.png"))
gImgPathL = sorted(glob.glob("/home/jovyan/nas-dataset/JUPR/MAM20044/Good/*.png"))
random.Random(7).shuffle( gImgPathL )

obj = Preprocess( outputFolder="../_data/mam_v1", mode="train", imgPathL1=bImgPathL,           imgPathL2=gImgPathL[:100], activeJsonPath=None)
obj = Preprocess( outputFolder="../_data/mam_v1", mode="eval" , imgPathL1=bImgPathL+gImgPathL, imgPathL2=gImgPathL[:100], activeJsonPath=None)

len(os.listdir(../_data/mam_v1/train/bad)) = 7
len(os.listdir(../_data/mam_v1/train/good)) = 99
len(os.listdir(../_data/mam_v1/eval/c0)) = 17765
len(os.listdir(../_data/mam_v1/eval/c1)) = 99
